In [4]:
import gym
import numpy as np

env = gym.make('CartPole-v0')

goal_average_steps = 195
max_number_of_steps = 200
num_consecutive_iterations = 100
num_episodes = 200
last_time_steps = np.zeros(num_consecutive_iterations)

for episode in range(num_episodes):
    # 環境の初期化
    observation = env.reset()

    episode_reward = 0
    for t in range(max_number_of_steps):
        # CartPoleの描画
        env.render()
        if env.viewer == None:
            env.render()
        # ランダムで行動の選択
        action = np.random.choice([0, 1])

        # 行動の実行とフィードバックの取得
        observation, reward, done, info = env.step(action)
        episode_reward += reward

        if done:
            print('%d Episode finished after %d time steps / mean %f' % (episode, t + 1,
                last_time_steps.mean()))
            last_time_steps = np.hstack((last_time_steps[1:], [episode_reward]))
            break

    if (last_time_steps.mean() >= goal_average_steps): # 直近の100エピソードが195以上であれば成功
        print('Episode %d train agent successfuly!' % episode)
        break

0 Episode finished after 40 time steps / mean 0.000000
1 Episode finished after 22 time steps / mean 0.400000
2 Episode finished after 15 time steps / mean 0.620000
3 Episode finished after 25 time steps / mean 0.770000
4 Episode finished after 19 time steps / mean 1.020000
5 Episode finished after 27 time steps / mean 1.210000
6 Episode finished after 25 time steps / mean 1.480000
7 Episode finished after 48 time steps / mean 1.730000
8 Episode finished after 10 time steps / mean 2.210000
9 Episode finished after 25 time steps / mean 2.310000
10 Episode finished after 26 time steps / mean 2.560000
11 Episode finished after 11 time steps / mean 2.820000
12 Episode finished after 9 time steps / mean 2.930000
13 Episode finished after 13 time steps / mean 3.020000
14 Episode finished after 10 time steps / mean 3.150000
15 Episode finished after 11 time steps / mean 3.250000
16 Episode finished after 19 time steps / mean 3.360000
17 Episode finished after 13 time steps / mean 3.550000
18 

In [1]:
import gym
import numpy as np

env = gym.make('CartPole-v0')

In [2]:
import numpy as np
q_table = np.random.uniform(low=-1, high=1, size=(4 ** 4, env.action_space.n))

def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]

def digitize_state(observation):
    # 各値を4個の離散値に変換
    cart_pos, cart_v, pole_angle, pole_v = observation
    digitized = [np.digitize(cart_pos, bins=bins(-2.4, 2.4, 4)),
                 np.digitize(cart_v, bins=bins(-3.0, 3.0, 4)),
                 np.digitize(pole_angle, bins=bins(-0.5, 0.5, 4)),
                 np.digitize(pole_v, bins=bins(-2.0, 2.0, 4))]
    # 0~255に変換
    return sum([x * (4 ** i) for i, x in enumerate(digitized)])

In [3]:
q_table

array([[ 1.42887377e-01,  3.42218576e-01],
       [-6.90537912e-01,  4.34317322e-01],
       [ 1.36129435e-01,  2.24759595e-01],
       [ 8.84897892e-01, -6.28865815e-01],
       [ 5.93203084e-01,  5.25986504e-01],
       [-4.94964814e-01,  5.50826278e-01],
       [ 4.97648203e-01, -6.84371204e-02],
       [-4.46509024e-01, -4.55312228e-01],
       [ 5.95078833e-01,  8.31879390e-01],
       [ 5.72426866e-01, -4.82941224e-01],
       [ 6.37123418e-01, -1.05435198e-01],
       [-4.58374358e-02,  6.95261591e-01],
       [ 1.60786354e-01,  1.37983003e-02],
       [ 8.73648900e-01, -5.11669718e-01],
       [-2.18889720e-01, -9.13011438e-01],
       [-5.05999416e-01,  7.87185238e-01],
       [ 4.10301607e-01,  2.50199704e-01],
       [ 1.42825667e-01, -3.25782681e-01],
       [-1.51699282e-01, -1.97202864e-01],
       [ 4.01697437e-01, -3.22802176e-01],
       [-9.61580130e-01, -4.41996724e-01],
       [-5.70121860e-01, -9.10178801e-01],
       [ 4.20274385e-01,  1.97452502e-01],
       [-6.

In [4]:
env.action_space.n

2

In [5]:
goal_average_steps = 195
max_number_of_steps = 200
num_consecutive_iterations = 100
num_episodes = 200
last_time_steps = np.zeros(num_consecutive_iterations)

In [6]:
def get_action(state, action, observation, reward):
    next_state = digitize_state(observation)
    next_action = np.argmax(q_table[next_state])

    # Qテーブルの更新
    alpha = 0.2
    gamma = 0.99
    q_table[state, action] = (1 - alpha) * q_table[state, action] +\
            alpha * (reward + gamma * q_table[next_state, next_action])

    return next_action, next_state

for episode in range(num_episodes):
    # 環境の初期化
    observation = env.reset()

    state = digitize_state(observation)
    action = np.argmax(q_table[state])

    episode_reward = 0
    for t in range(max_number_of_steps):
        # CartPoleの描画
        #env.render()

        # 行動の実行とフィードバックの取得
        observation, reward, done, info = env.step(action)

        # 行動の選択
        action, state = get_action(state, action, observation, reward)
        episode_reward += reward

C:\Users\lllni\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


KeyboardInterrupt: 

In [7]:
q_table[0]

array([-0.43490312, -0.13428966])

In [8]:
action, state = get_action(state, action, observation, reward)
print(action)
print(state)

0
9
